In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import pyodbc
import os

In [6]:
f = open("place_sum_tem_1.csv",encoding='utf-8')
place = pd.read_csv(f)

In [7]:
want_place = np.array(place.CASE_NO)

In [4]:
server = '168.14.xx.xx'
username = 'username'
password = 'password'
database = 'database name'
driver = '{ODBC Driver 13 for SQL Server}'
connectionString = 'DRIVER={0};PORT=1433;SERVER={1};DATABASE={2};UID={3};PWD={4}'.format(driver, server, database, username, password)
cnxn = pyodbc.connect(connectionString)
cursor = cnxn.cursor()
instruct_day='''
SELECT [ID] ,[CITY_ID],[DISTRICT_ID],[REGION_ID],[KWP],[DATA_TIME],[SUN],[POWER],[TEMPERATURE]
FROM [dbname].[dbo].[power]
WHERE [DATA_TIME] BETWEEN '2017-09-01 00:00:00' and '2018-05-31 21:00:00'
'''

df_day=pd.read_sql(instruct_day, cnxn)

In [7]:
len(df_day)

11109177

In [27]:
d = np.array(df_day["DISTRICT_ID"])
r = np.array(df_day["REGION_ID"])
d_r = []
for i in range(len(d)):
    dd = d[i].split("D")
    dd = dd[1]
    rr = r[i].split("R")
    rr = rr[1]
    dr = dd + rr
    d_r.append(dr)
d_r = pd.DataFrame(d_r,columns=["CASE_ID"])
df_day_1 = pd.concat([df_day,d_r],axis=1)

In [42]:
df_day_1.CASE_NO = df_day_1.CASE_NO.astype(str)

In [51]:
data = pd.DataFrame(columns=['ID', 'CITY_ID', 'DISTRICT_ID', 'REGION_ID', 'KWP', 'DATA_TIME',
       'SUN', 'POWER', 'TEMPERATURE', 'CASE_ID'])
for i in want_place:
    i = str(i)
    a = df_day_1[df_day_1.CASE_NO == i]
    data = pd.concat([data,a],axis=0)

In [52]:
data['DATA_TIME_datetype'] = pd.to_datetime(data['DATA_TIME'])
data = data[["CITY_ID","CASE_ID","DATA_TIME_datetype",'SUN', 'POWER', 'TEMPERATURE','KWP']]
data = data.reset_index(drop=True)

In [53]:
day_time = np.array(data.DATA_TIME_datetype)
day = []
time = []
for i in range(len(day_time)):
    dt = str(day_time[i])
    dt = dt.split("T")
    day.append(dt[0])
    ti = dt[1]
    ti = ti[0:5]
    time.append(ti)
day_df = pd.DataFrame(day,columns=["day"])
time_df = pd.DataFrame(time,columns=["time"])
df_1 = pd.concat([data,day_df,time_df],axis=1)

In [14]:
case_list1 = pd.unique(df_1.CASE_ID)
caselist = sorted(case_list1)

day_case_W = []

for i in caselist:
    da = df_1[df_1.CASE_NO == i]
    da_1 = da.sort_values(by=['DATA_TIME_datetype'])
    da_1['TOTAL_SUN'] = da_1['SUN'].cumsum()
    daylist1 = pd.unique(da.day)
    daylist = sorted(daylist1)
    for l in daylist:
        da_2 = da_1[da_1.day==l]
        if len(da_2) != 96 :
            day_case_W.append([i,l])

In [10]:
df_1 = df_1[df_1.day != "2018-05-31"]

In [4]:
df_2 = pd.DataFrame(columns=['CITY_ID', 'CASE_ID', 'DATA_TIME_datetype', "SUN","POWER", 'TEMPERATURE', 
                             'KWP','day', 'time',"TOTAL_SUN",'final_sun', 'final_kwh',"tem_3hr_mean"])

In [5]:
case_list1 = pd.unique(df_1.CASE_ID)
caselist = sorted(case_list1)
for i in caselist:
    da = df_1[df_1.CASE_ID == i]
    da_1 = da.sort_values(by=['DATA_TIME_datetype'])
    da_1['TOTAL_SUN'] = da_1['SUN'].cumsum()
    daylist1 = pd.unique(da_1.day)
    daylist = sorted(daylist1)
    for l in daylist:
        tem_3hr_mean = []
        da_2 = da_1[da_1.day==l]
        tem = np.array(da_2.TEMPERATURE)
        a = [0,12,24,36,48,60,72,84]
        for m in a:
            for n in range(m+12):
                tem_3hr_list = []
                tem_3hr_list.append(tem[n])
            tem_mean = np.mean(tem_3hr_list)
            tem_3hr_mean.append(tem_mean)
        tem_3hr_mean = pd.DataFrame(tem_3hr_mean,columns=["tem_3hr_mean"]) 
    
        da_3 = da_2.loc[(da_2['time'] == '00:00') | (da_2['time'] == '03:00') |
                (da_2['time'] == '06:00') | (da_2['time'] == '09:00') |
                (da_2['time'] == '12:00') | (da_2['time'] == '15:00') |
                (da_2['time'] == '18:00') | (da_2['time'] == '21:00')]
        da_3 = da_3.reset_index(drop=True)
        da_3['final_sun'] = da_3['TOTAL_SUN'].shift(-1) - da_3['TOTAL_SUN']
        da_3['final_kwh'] = da_3['POWER'].shift(-1) - da_3['POWER']
        da_4 = pd.concat([da_3,tem_3hr_mean],axis=1)
        df_2 = pd.concat([df_2,da_4],axis=0)

In [24]:
def clean_data(data):
    global day_W
    global day_C
    global df_new
    day_C = []
    day_W = []
    caselist_1 = pd.unique(data.CASE_NO)
    caselist = sorted(caselist_1)
    for i in caselist:
        da_1 = data[data.CASE_NO == i]
        daylist_1 = pd.unique(da_1.day)
        daylist = sorted(daylist_1)
        for l in daylist:
            da_2 = da_1[da_1.day == l]
            sun = np.array(da_2.final_sun)
            kwh = np.array(da_2.final_kwh)
            if min(sun) < 0:
                day_W.append([i,l])
            elif min(kwh) < 0:
                day_W.append([i,l])
            else :
                day_C.append([i,l])
    df_new = pd.DataFrame(columns=['CITY_ID', 'CASE_NO', 'DATA_TIME_datetype', 'SUN', 'POWER',
       'TEMPERATURE', 'KWP', 'day', 'time', 'TOTAL_SUN', 'final_sun','final_kwh', 'tem_3hr_mean'])
    for m in day_C:
        da_3 = data[(data.CASE_NO==m[0]) & (data.day==m[1])]
        df_new = pd.concat([df_new,da_3],axis=0)

In [25]:
clean_data(df_2)
df_3 = df_new

In [18]:
len(df_3)

0

In [19]:
df_4 = df_3.loc[(df_3['time'] == '06:00') | (df_3['time'] == '09:00')|
                (df_3['time'] == '12:00') | (df_3['time'] == '15:00')]

In [21]:
df_4.to_csv("df_4_0622.csv",index=False)

In [23]:
def clean_data_tem(data):
    global day_W
    global day_C
    global df_new
    day_C = []
    day_W = []
    caselist_1 = pd.unique(data.CASE_ID)
    caselist = sorted(caselist_1)
    for i in caselist:
        da_1 = data[data.CASE_ID == i]
        daylist_1 = pd.unique(da_1.day)
        daylist = sorted(daylist_1)
        for l in daylist:
            da_2 = da_1[da_1.day == l]
            tem = np.array(da_2.tem_3hr_mean)
            if min(tem) == 0:
                day_W.append([i,l])
            else :
                day_C.append([i,l])
    df_new = pd.DataFrame(columns=['CITY_ID', 'CASE_ID', 'DATA_TIME_datetype', 'SUN', 'POWER',
       'TEMPERATURE', 'KWP', 'day', 'time', 'TOTAL_SUN', 'final_sun','final_kwh', 'tem_3hr_mean'])
    for m in day_C:
        da_3 = data[(data.CASE_ID==m[0]) & (data.day==m[1])]
        df_new = pd.concat([df_new,da_3],axis=0)

In [24]:
clean_data_tem(df_4)
df_5 = df_new

In [30]:
server = '168.14.xx.xx'
username = 'username'
password = 'password'
database = 'database name'
driver = '{ODBC Driver 13 for SQL Server}'
connectionString = 'DRIVER={0};PORT=1433;SERVER={1};DATABASE={2};UID={3};PWD={4}'.format(driver, server, database, username, password)
cnxn = pyodbc.connect(connectionString)
cursor = cnxn.cursor()


site="SELECT [CITY_ID],[CITY_NAME],[DISTRICT_ID],[DISTRICT_NAME],[REGION_ID],[CASE_NO],[CASE_NAME],[KWP],[THERMOMETER_AMOUNT],[SUNSHINEMETER_AMOUNT],[ENABLE_DATE] FROM  [dbname].[dbo].[site]"
df_site=pd.read_sql(site, cnxn)
df_site = df_site[[ 'CITY_NAME', 'DISTRICT_NAME','CASE_ID', 'CASE_NAME']]

In [31]:
cursor = cnxn.cursor()
cwb = '''
SELECT [CITY],[DISTRICT],[GEOCODE],[DAYTIME],[T],[TD],[RH],[WD],[WS],[BF],[AT],[Wx],[Wx_n],[PoP6h],[PoP12h],[get_day]
FROM [dbname].[dbo].[WEATHER_CRAWLER]
'''
df_cwb=pd.read_sql(cwb, cnxn)

# split daytime
a = np.array(df_cwb.DAYTIME)
day = []
time = []
for x in a:
    x = str(x)
    x = x.split("T")
    day.append(str(x[0]))
    b = x[1].split(".")
    time.append(str(b[0]))
day = pd.DataFrame(day,columns=["DAY"])
time = pd.DataFrame(time,columns=["TIME"])
xx = pd.concat([day,time],1)
df_cwb = pd.concat([df_cwb,xx],1)
df_cwb = df_cwb[['CITY', 'DISTRICT', 'GEOCODE', 'DAYTIME','DAY','TIME', 'T', 'TD', 'RH', 'WD', 'WS','BF', 'AT', 'Wx', 'Wx_n', 'PoP6h', 'PoP12h', 'get_day']]
df_cwb.Wx[df_cwb.Wx_n == 1] ="晴天"

o = []
ot = np.array(df_cwb.TIME)
for i in ot:
    otime = i.split(":")[0]
    o.append(int(otime))
o = pd.DataFrame(o,columns=['otime'])
df_cwb_o = pd.concat([df_cwb,o],axis = 1)

df_cwb_9_18 = df_cwb_o[(df_cwb_o.otime >= 6 )& (df_cwb_o.otime <= 15)]
df_cwb_9_18 = df_cwb_9_18.drop_duplicates()  
df_cwb_9_18 = df_cwb_9_18.reset_index(drop=True)

In [32]:
df_site.CASE_NO = df_site.CASE_NO.astype(str)
df_5.CASE_NO = df_5.CASE_NO.astype(str)
df_m = pd.merge(df_5,df_site,on=["CASE_ID"])
df_cwb_9_18 = df_cwb_9_18[['CITY', 'DISTRICT', 'GEOCODE', 'DAYTIME', 'DAY', 'TIME', 'T', 'TD','RH', 
                           'WD', 'WS', 'BF', 'AT', 'Wx', 'Wx_n', 'PoP6h', 'PoP12h', 'get_day']]
df_cwb_9_18.columns = ['CITY_NAME', 'DISTRICT_NAME', 'GEOCODE', 'DATA_TIME_datetype', 'DAY', 'TIME', 'T', 'TD',
       'RH', 'WD', 'WS', 'BF', 'AT', 'Wx', 'Wx_n', 'PoP6h', 'PoP12h','get_day']
df_cwb_9_18.CITY_NAME[df_cwb_9_18.CITY_NAME == "臺北市"] = "台北市"
df_cwb_9_18.CITY_NAME[df_cwb_9_18.CITY_NAME == "臺中市"] = "台中市"
df_cwb_9_18.CITY_NAME[df_cwb_9_18.CITY_NAME == "臺南市"] = "台南市"

C:\Users\csim\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\csim\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [57]:
df_m['DATA_TIME_datetype'] = pd.to_datetime(df_m['DATA_TIME_datetype'])
df_cwb_9_18['DATA_TIME_datetype'] = pd.to_datetime(df_cwb_9_18['DATA_TIME_datetype'])

In [58]:
df_m_cwb  = pd.merge(df_m,df_cwb_9_18,on=["DATA_TIME_datetype","CITY_NAME","DISTRICT_NAME"])

In [60]:
df_m_cwb = df_m_cwb.sort_values(by=['CASE_NO','DATA_TIME_datetype']) 
df_m_cwb = df_m_cwb.reset_index(drop=True)

In [42]:
def data_select(data,we_day):
    global df_new
    df_new = pd.DataFrame(columns=['CITY_ID', 'CASE_ID', 'DATA_TIME_datetype', 'PV_TOTAL_SUN', 'TOTAL_KWH',
                                   'PV_TOTAL_TEMPERATURE', 'KWP', 'day', 'time', 'TOTAL_SUN', 'final_sun',
                                   'final_kwh', 'tem_3hr_mean', 'CITY_NAME', 'DISTRICT_NAME', 'CASE_NAME',
                                   'GEOCODE', 'DAY', 'TIME', 'T', 'TD', 'RH', 'WD', 'WS', 'BF', 'AT', 'Wx',
                                   'Wx_n', 'PoP6h', 'PoP12h', 'get_day'])
    for m in we_day:
        da_3 = data[(data.CASE_NO==m[0]) & (data.day==m[1])]
        df_new = pd.concat([df_new,da_3],axis=0)

def select_sun(data):
    global sun_day
    sun_day = []
    global no_sun_day
    no_sun_day = []
    global no_sun_data
    global sun_data
    no_sun_data = pd.DataFrame(columns=['CITY_ID', 'CASE_NO', 'DATA_TIME_datetype', 'PV_TOTAL_SUN', 'TOTAL_KWH',
                                   'PV_TOTAL_TEMPERATURE', 'KWP', 'day', 'time', 'TOTAL_SUN', 'final_sun',
                                   'final_kwh', 'tem_3hr_mean', 'CITY_NAME', 'DISTRICT_NAME', 'CASE_NAME',
                                   'GEOCODE', 'DAY', 'TIME', 'T', 'TD', 'RH', 'WD', 'WS', 'BF', 'AT', 'Wx',
                                   'Wx_n', 'PoP6h', 'PoP12h', 'get_day'])
    sun_data = pd.DataFrame(columns=['CITY_ID', 'CASE_NO', 'DATA_TIME_datetype', 'PV_TOTAL_SUN', 'TOTAL_KWH',
                                   'PV_TOTAL_TEMPERATURE', 'KWP', 'day', 'time', 'TOTAL_SUN', 'final_sun',
                                   'final_kwh', 'tem_3hr_mean', 'CITY_NAME', 'DISTRICT_NAME', 'CASE_NAME',
                                   'GEOCODE', 'DAY', 'TIME', 'T', 'TD', 'RH', 'WD', 'WS', 'BF', 'AT', 'Wx',
                                   'Wx_n', 'PoP6h', 'PoP12h', 'get_day'])
    caselist1=pd.unique(data['CASE_NO'])
    caselist=sorted(caselist1)
    for i in caselist:
        da = data[data.CASE_NO==i]
        daylist1=pd.unique(da['DAY'])
        daylist=sorted(daylist1)
        for l in daylist:
            da_1 = da[da.DAY==l]
            a = np.array(da_1.Wx_n)
            if max(a) > 2:
                da_3 = data[(data.CASE_NO==i) & (data.day==l)]
                no_sun_data = pd.concat([no_sun_data,da_3],axis=0)
                no_sun_day.append([i,l])
            else :
                da_3 = data[(data.CASE_NO==i) & (data.day==l)]
                sun_data = pd.concat([sun_data,da_3],axis=0)
                sun_day.append([i,l])

In [43]:
select_sun(df_m_cwb)

In [44]:
no_sun_data = no_sun_data.reset_index(drop=True)
sun_data = sun_data.reset_index(drop=True)

In [48]:
no_sun_data_1 = no_sun_data.drop_duplicates(subset=['CASE_NO',"DATA_TIME_datetype"],keep="last")
sun_data_1 = sun_data.drop_duplicates(subset=['CASE_NO',"DATA_TIME_datetype"],keep="last")

In [65]:
no_sun_data = no_sun_data_1.reset_index(drop=True)
sun_data = sun_data_1.reset_index(drop=True)

In [66]:
kwh_x =[]
caselist1=pd.unique(sun_data['CASE_NO'])
caselist=sorted(caselist1)
for i in caselist:
    data = sun_data[sun_data.CASE_NO==i]
    daylist1=pd.unique(data['DAY'])
    daylist=sorted(daylist1)
    for l in daylist:
        da = data[data.DAY==l]
        kwh = np.array(da.final_kwh)
        kwh_x.append(0)
        kwh_x.append(kwh[0])
        kwh_x.append(kwh[1])
        kwh_x.append(kwh[2])
kwh_x = pd.DataFrame(kwh_x,columns=["kwh_x"])
sun_data_new = pd.concat([sun_data,kwh_x],axis=1)

In [68]:
kwh_x =[]
caselist1=pd.unique(no_sun_data['CASE_NO'])
caselist=sorted(caselist1)
for i in caselist:
    data = no_sun_data[no_sun_data.CASE_NO==i]
    daylist1=pd.unique(data['DAY'])
    daylist=sorted(daylist1)
    for l in daylist:
        da = data[data.DAY==l]
        kwh = np.array(da.final_kwh)
        kwh_x.append(0)
        kwh_x.append(kwh[0])
        kwh_x.append(kwh[1])
        kwh_x.append(kwh[2])
kwh_x = pd.DataFrame(kwh_x,columns=["kwh_x"])
no_sun_data_new = pd.concat([no_sun_data,kwh_x],axis=1)

In [70]:
sun_data_new.to_csv("sun_data_0624.csv",index=False,encoding="utf_8_sig")
no_sun_data_new.to_csv("no_sun_data_0624.csv",index=False,encoding="utf_8_sig")